# TP 4 RNNs: classification de séquences, forecasting, génération de séquences (suite)

_Ismaël Bonneau_

Ce notebook sert uniquement à présenter nos résultats, et les bouts de code intéressants dans le cadre de ce rapport. L'intégralité du code est contenu dans le fichier tp4.py.

## But

Nous avons dans le notebook précédent codé un RNN from scratch et nous l'avons appliqué sur une tâche de classification de séquence. Nous allons maintenant reprendre ce RNN et réaliser une tâche de génération de séquence.

## Données

Le jeu de données suggéré dans l'énoncé est un discours de Donald Trump. Cependant, il est beaucoup plus drôle de travailler sur le script du seigneur des anneaux. 

<img src="boromir.jpg">

In [1]:
import numpy as np
import torch
import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

%matplotlib inline
%load_ext autoreload
%autoreload 2

import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
bestmovieever = pd.read_csv("data/lord-of-the-rings-data/lotr_scripts.csv")

### Récupérons les lignes de dialogue des 3 films du seigneur des anneaux. Nous allons garder uniquement les répliques de Gandalf.

In [3]:
bestmovieever[bestmovieever["char"] == "GANDALF"].head()

,Unnamed: 0,char,dialog,movie
34,34,GANDALF,Now come the days of the King. May they be...,The Return of the King
68,68,GANDALF,Hobbits!,The Return of the King
72,72,GANDALF,"Be careful. Even in defeat, Saruman is dangero...",The Return of the King
74,74,GANDALF,"No, we need him alive. We need him to talk.",The Return of the King
78,78,GANDALF,Your treachery has already cost many lives. Th...,The Return of the King


### On nettoie un peu les lignes de dialogue:

In [5]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_tags
import re

CUSTOM_FILTERS = [strip_tags, strip_multiple_whitespaces]

hey = list(bestmovieever[bestmovieever["char"] == "GANDALF"]["dialog"])
gandalf = []
for d in hey:
    mdr = " ".join(preprocess_string(d, CUSTOM_FILTERS))
    for ahaha in mdr.split("."):
        if ahaha != "":
            x = ahaha.strip()+"."
            x = re.sub(r'^ , ', '', x)
            x = re.sub(r'^, ', '', x)
            x = re.sub(r'^,', '', x)
            gandalf.append(x)

### Nous allons travailler au niveau des caractères. 

il faut encoder chaque caractère sous forme d'id numérique (entier).

In [6]:
import string
import unicodedata

max_len = len(gandalf[np.argmax([len(s) for s in gandalf])])
print("longueur max. de sequence: {}".format(max_len))

# use $ as a end-of-sequence character and @ as a start-of-sequence
LETTRES = string.ascii_letters + '!"&\'(),-./:;?[\\]' + ' '
id2lettre = dict(zip(range(1, len(LETTRES)+1),LETTRES))
id2lettre[0]= '' ##NULL CHARACTER
lettre2id = dict(zip(id2lettre.values(), id2lettre.keys()))

def normalize(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if c in LETTRES)
def string2code(s):
    base = [lettre2id[c] for c in normalize(s)]
    if len(base) < max_len:
        padding = [0] * (max_len - len(base))
    else:
        padding = []
    return base + padding + [0]
def code2string(t):
    if type(t) != list:
        t = t.tolist()
    return ''.join(id2lettre[i] for i in t)

longueur max. de sequence: 187


In [7]:
print(gandalf[0])
print(string2code(gandalf[0]))

Now come the days of the King.
[40, 15, 23, 69, 3, 15, 13, 5, 69, 20, 8, 5, 69, 4, 1, 25, 19, 69, 15, 6, 69, 20, 8, 5, 69, 37, 9, 14, 7, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### Passage en one-hot encoding:

In [8]:
from sklearn.preprocessing import OneHotEncoder

data = np.array([np.array(string2code(s)) for s in gandalf])
print("avant one-hot encoding: ", data.shape)

avant one-hot encoding:  (424, 188)


In [9]:
one_hot_encoded = []
for elt in data:
    onehotencoder = OneHotEncoder(sparse=False, categories=[range(len(LETTRES)+1)])
    one_hot_encoded.append(onehotencoder.fit_transform(elt.reshape(-1, 1)))
one_hot_encoded = np.array(one_hot_encoded)
print("dimensions maintenant: ", one_hot_encoded.shape)

dimensions maintenant:  (424, 188, 70)


Le RNN que nous avons codé attend en entrée des matrices de taille sequence_length x batch x dim. On va donc swaper les dimensions 0 et 1 de notre array de séquences:

In [10]:
X_train = np.swapaxes(one_hot_encoded, 0, 1)
print(X_train.shape)

(188, 424, 70)


## Modèle

Cette fois-ci, on introduit la supervision à chaque étape: au lieu de devoir encoder correctement une classe dans l'état ${h_T}$, le modèle doit réussir à produire ${h_t}$ à partir de ${h_{t-1}}$ à chaque étape. Chaque "cellule" du RNN déplié doit donc appliquer un décodeur sur l'état ${h_{t-1}}$ qu'elle reçoit en entrée.

Le fait que l'on utilise des caractères rajoute une difficulté supplémentaire: il va falloir en même temps calculer une projection des caractères dans un espace continu (embedding).

In [43]:
class RNN(torch.nn.Module):
    """docstring for RNN"""
    def __init__(self, dim, latent):
        super(RNN, self).__init__()
        self.dim = dim
        self.latent = latent
        self.Wx = torch.nn.Linear(dim, latent)
        self.Wh = torch.nn.Linear(latent, latent)

    def forward(self, x, h=None):
        """ x: sequence_length x batch x dim 
            h: batch x latent
            returns: length x batch x latent Tensor"""
        historique = []
        # pour chaque instant de la sequence:
        if h is None:
            ht = torch.zeros(x.size()[1], self.latent)
            historique.append(ht)
        for i, xt in enumerate(x):
            # ht: (batch x latent)
            ht = self.one_step(xt, ht)
            historique.append(ht) # Ne pas enregistrer les h0
        return historique

    def one_step(self, x, h):
        """ x: batch x dim 
            h: batch x latent
            returns: batch x latent Tensor """
        return torch.nn.functional.leaky_relu(self.Wx(x) + self.Wh(h))

In [50]:
class Decoder_cell(torch.nn.Module):
    """ decode un etat caché """
    def __init__(self, latent, dim):
        super(Decoder_cell, self).__init__()
        self.W = nn.Sequential(nn.Linear(latent, 16),
                               nn.Tanh(),
                               nn.Linear(16, dim), nn.Softmax(dim=2))
        # doit pouvoir produire une classe à partir d'un état caché
        
    def forward(self, h):
        """ """
        return self.W(h)

In [51]:
rnn = RNN(X_train.shape[2], 32)

In [52]:
h = torch.stack(rnn(torch.from_numpy(X_train).float()))

In [53]:
decoder = Decoder_cell(32, X_train.shape[2])

In [54]:
khabib = decoder(h)

In [55]:
khabib.size()

torch.Size([189, 424, 70])

In [58]:
khabib[1:,:,:].size()

torch.Size([188, 424, 70])

In [56]:
criterion = torch.nn.CrossEntropyLoss()

In [57]:
criterion(khabib[1:,:,:], X_train)

TypeError: 'int' object is not callable